# What Require Inputs in this Notebook
1. [Fill in File Input](#Load-in-Data)
2. [Right or Left Hand](#Right-or-Left-Hand)
3. [Output ShotCSV](#Output-ShotCSV)
4. [Change to atNet (by Player) from tagger later](#At-Net)
5. [EDA and get Stats Here](#EDA)
6. [Output PointCSV for Visuals](#PRINT-POINTCSV-VISUALS)
7. [Output PointCSV for Upload](#Print-POINTCSV-UPLOAD)

# Table of Contents
1. [Load in Data](#Load-in-Data)
- Error Checking
- Add Columns
- [Output ShotCSV](#Output-ShotCSV)
2. [Create Point DF](#Create-Point-DF)
- Add Columns
- [Output PointCSV for Visuals](#PRINT-POINTCSV-VISUALS)
- [Output PointCSV for Upload](#Print-POINTCSV-UPLOAD)    
        - cut out points with no timestamp position    
        - atNetPlayer1 has values of the player name for display purposes instead of boolean values

# Need to change soon
- atNetPlayer1, atNetPlayer2 are being aggregated by if hitting a volley. Will be replaced by tagger values for those
- pointWonBy is being calculated, replaced by tagger soon
- player1Hand and player2Hand are being manually inputted

# Add
- More stats on approach shots
- Pressure Point stats, like what did players did on 30-30 points onwards
- Inside Out/Inside In Stats once we get matches with accurate side info
- handling of rows where there is error on video side and the point is cut short

Notes:
- Keep empty values None Type, they will be converted to empty string "" right before outputting to Point CSV Only for Upload
- Having 1 does not mean other value is boolean 0, it is None

# Post Process
- Adds Point Number (Comment out if already there)
- Forward Fills these columns'gameScore','setScore', 'tiebreakScore', 'serverName', 'player1Name', 'player2Name'
   - Add this when in tagger: player1Hand, player2Hand 
- Adds Returner Name
- Adds Shot Hit By for each Shot
- Adds isInsidein/isInsideOut (booleans 0/1) Columns

In [1064]:
import pandas as pd
import numpy as np

## Load in Data

In [1065]:
# Put your file name here

your_file_name = "filename.csv"
shot_data = pd.read_csv(your_file_name)



# Fill in Player Name

In [1066]:
if shot_data.at[0, 'player1Name'] == None:
    print("player1Name was blank. Filling with 'fillInplayer1Name'")
    shot_data.at[0, 'player1Name'] = 'fillInplayer1Name'
else:
    print("player1Name is not blank. Current value:", shot_data.at[0, 'player1Name'])

# Check if player2Name is blank, if so, fill it with 'fillInplayer2Name'
if shot_data.at[0, 'player2Name'] == None:
    print("player2Name was blank. Filling with 'fillInplayer2Name'")
    shot_data.at[0, 'player2Name'] = 'fillInplayer2Name'
else:
    print("player2Name is not blank. Current value:", shot_data.at[0, 'player2Name'])


player1Name is not blank. Current value: Kimmi Hance
player2Name is not blank. Current value: Malaika Rapolu


- add side and pointNumber Columns

In [1067]:
# # IF NO SIDE Column (Old tagger)
# deuce_values = ['0-0', '15-15', '40-15', '30-0', '0-30', '15-40', '40-40 (Deuce)', '30-30']
# shot_data['side'] = shot_data.apply(lambda row: 'Deuce' if str(row['pointScore']) in deuce_values and row['isPointStart'] == 1 else 'Ad' if str(row['pointScore']) is not None and row['isPointStart'] == 1 else None, axis=1)


In [1068]:
 # Check if both conditions passed
if 'pointNumber' in shot_data.columns and not shot_data['pointNumber'].isnull().any() and shot_data['pointNumber'].is_monotonic_increasing:
    print("All conditions passed: 'pointNumber' column exists for every row, is not empty, and is in increasing order.")


# Check if pointNumber exists for every row and is not empty
if 'pointNumber' not in shot_data.columns or shot_data['pointNumber'].isnull().any():
    # Execute the script to calculate pointNumber
    point_starts = (shot_data['isPointStart'] == 1)
    shot_data['pointNumber'] = point_starts.cumsum()
    
    print("Condition 1 failed: 'pointNumber' column does not exist for every row or contains empty values.")
    
# Check if pointNumber is in increasing order
if not shot_data['pointNumber'].is_monotonic_increasing:
    # Print the condition failure if the 'pointNumber' column is not in increasing order
    print("Condition 2 failed: 'pointNumber' column is not in increasing order.")
    point_starts = (shot_data['isPointStart'] == 1)
    shot_data['pointNumber'] = point_starts.cumsum()
   

Condition 1 failed: 'pointNumber' column does not exist for every row or contains empty values.


In [1069]:
missing_rows = shot_data[shot_data['pointNumber'].isnull()]
print(f"Count of rows missing 'pointNumber': {len(missing_rows)}")
# print(missing_rows)

Count of rows missing 'pointNumber': 0


- Check for NA's (Revise Data if needed)
- shotInRally
- pointScore
- gameScore
- setScore
- side
- pointStartTime when isPointStart = 1
- pointEndTime when sPointEnd = 1

In [1070]:
# Count empty strings in each column
empty_string_counts = (shot_data == "").sum()

# Filter out columns with zero empty strings
non_zero_counts = empty_string_counts[empty_string_counts > 0]

# Display the counts
print("Count of empty strings in each column that includes at least one:")
print(non_zero_counts)
# Count NaN or empty values in specified columns
na_counts = shot_data[['shotInRally', 'pointScore', 'gameScore', 'setScore', 'side', 'player1Name', 'player2Name','serverName']].isna().sum()

# Display the counts
print(f"\nCout of NA's in these columns\n{na_counts}")

# Count NaN values in 'pointScore' column when 'gameScore' is '6-6'
na_count = shot_data.loc[shot_data['gameScore'] == '6-6', 'pointScore'].isna().sum()

# Display the count
print("\nCount of NaN values in 'pointScore' when 'gameScore' is '6-6':", na_count)

# Count NaN values in 'pointScore' column when 'gameScore' is '6-6'
side_na_count = shot_data.loc[shot_data['isPointStart'] == 1, 'side'].isna().sum()

# Display the count
print("\nCount of NaN values for side on Point Start:", side_na_count)



Count of empty strings in each column that includes at least one:
Series([], dtype: int64)

Cout of NA's in these columns
shotInRally      0
pointScore     137
gameScore        0
setScore         0
side           330
player1Name    978
player2Name    978
serverName       0
dtype: int64

Count of NaN values in 'pointScore' when 'gameScore' is '6-6': 137

Count of NaN values for side on Point Start: 0


In [1071]:
empty_shot_rows = shot_data[shot_data['shotInRally'].isnull()]

# Iterate over the index of empty_shot_rows
for index in empty_shot_rows.index:
    # Get the row with empty shotInRally
    empty_row = shot_data.loc[index]
    
    # Get the row above it
    if index - 1 >= 0:
        above_row = shot_data.loc[index - 1]
    else:
        above_row = None
    
    # Get the row below it
    if index + 1 < len(shot_data):
        below_row = shot_data.loc[index + 1]
    else:
        below_row = None
    
    # Print the rows
    print("Empty row:")
    print(empty_row)
    print("Row above:")
    print(above_row)
    print("Row below:")
    print(below_row)
    print("====================")


In [1072]:
# Find rows where isPointStart is True, but missing pointStartTime
missing_start_time = shot_data[((shot_data['isPointStart'] == 1) & shot_data['pointStartTime'].isna())]
print(f"Count of rows where PointStartTime is missing: {len(missing_start_time)}")


print(f"These are rows where PointStartTime is missing \n{missing_start_time}")

Count of rows where PointStartTime is missing: 3
These are rows where PointStartTime is missing 
    pointScore gameScore setScore  isPointStart  pointStartTime  isPointEnd  \
249       0-40       2-3      0-0           1.0             NaN         NaN   
339      30-30       3-5      0-0           1.0             NaN         1.0   
645       0-15       4-3      0-1           1.0             NaN   5412611.0   

     pointEndTime  shotInRally   side      serverName  ...  lineupPosition  \
249           NaN            6     Ad     Kimmi Hance  ...             NaN   
339           NaN            1  Deuce  Malaika Rapolu  ...             NaN   
645           NaN            1     Ad     Kimmi Hance  ...             NaN   

    player1Name  player2Name  Round  Surface Notes  shotDirection  shotFhBh  \
249         NaN          NaN    NaN      NaN   NaN            NaN       NaN   
339         NaN          NaN    NaN      NaN   NaN            NaN       NaN   
645         NaN          NaN    NaN 

In [1073]:
# Find rows where isPointEnd is True, but missing pointEndTime
missingEndTime = shot_data[((shot_data['isPointEnd'] == 1) & shot_data['pointEndTime'].isna())]
print(f"Count of rows where PointEndTime is missing: {len(missingEndTime)}")

# Display the missing rows 
print(f"These are rows where PointEndTime is missing \n{missingEndTime}")

Count of rows where PointEndTime is missing: 2
These are rows where PointEndTime is missing 
    pointScore gameScore setScore  isPointStart  pointStartTime  isPointEnd  \
339      30-30       3-5      0-0           1.0             NaN         1.0   
970        NaN       6-6      0-1           NaN             NaN         1.0   

     pointEndTime  shotInRally   side      serverName  ...  lineupPosition  \
339           NaN            1  Deuce  Malaika Rapolu  ...             NaN   
970           NaN           11  Deuce  Malaika Rapolu  ...             NaN   

    player1Name  player2Name  Round  Surface Notes  shotDirection  shotFhBh  \
339         NaN          NaN    NaN      NaN   NaN            NaN       NaN   
970         NaN          NaN    NaN      NaN   NaN  Down the Line  Forehand   

     tiebreakScore  returnData  
339            NaN         NaN  
970            7-8         NaN  

[2 rows x 50 columns]


- Make Jan-00 back into 1-0 for Game/Set Score

In [1074]:
# Make Scores Strings not Date Time
columns_to_convert = ['gameScore', 'setScore'] #if no tiebreakScore
# columns_to_convert = ['gameScore', 'setScore', 'tiebreakScore']
shot_data[columns_to_convert] = shot_data[columns_to_convert].astype(object)

In [1075]:
import re

# Define a mapping for month abbreviations
month_mapping = {'Jan': '1', 'Feb': '2', 'Mar': '3', 'Apr': '4', 'May': '5', 'Jun': '6',
                 'Jul': '7', 'Aug': '8', 'Sep': '9', 'Oct': '10', 'Nov': '11', 'Dec': '12'}

# Function to convert string like 'Jan-00' to '1-0'
def convert_score_string(score_str):
    # Check if the string has a month abbreviation and a year ending with '00'
    if re.match(r'^\d{1,2}-[A-Za-z]{3}$', score_str):
        # Extract year and month abbreviation
        year, month = score_str.split('-')

        # Remove leading zeros from the year
        year = str(int(year))

        # Replace month abbreviation with corresponding number
        month_number = month_mapping.get(month, month)

        # Concatenate the parts to form the transformed string
        transformed_str = f'{year}-{month_number}'

        return transformed_str

    # Check if the string has a month abbreviation and a year with leading '0's
    elif re.match(r'^[A-Za-z]{3}-\d{1,2}$', score_str):
        # Extract month abbreviation and year
        month, year = score_str.split('-')

        # Replace month abbreviation with corresponding number
        month_number = month_mapping.get(month, month)

        # Remove leading zeros from the year
        year = str(int(year))

        # Concatenate the parts to form the transformed string
        transformed_str = f'{month_number}-{year}'

        return transformed_str

    return score_str

# Apply the conversion function to the relevant columns in shot_data
shot_data['gameScore'] = shot_data['gameScore'].apply(convert_score_string)
shot_data['setScore'] = shot_data['setScore'].apply(convert_score_string)
# shot_data['tiebreakerScore'] = shot_data['tiebreakerScore'].apply(convert_score_string)


In [1076]:
# Assuming shot_data is your DataFrame
unique_set_scores = shot_data['setScore'].unique()
unique_game_scores = shot_data['gameScore'].unique()

# Print unique values
print("Unique Set Scores:", unique_set_scores)
print("Unique Game Scores:", unique_game_scores)


Unique Set Scores: ['0-0' '0-1']
Unique Game Scores: ['0-0' '1-0' '1-1' '1-2' '2-2' '2-3' '2-4' '3-4' '3-5' '4-5' '2-0' '2-1'
 '3-2' '3-3' '4-3' '5-3' '5-4' '5-5' '5-6' '6-6']


In [1077]:
# Create new columns
import pandas as pd

# Assuming shot_data is your DataFrame

# Define functions to extract shotFhBh and shotDirection from shotType
def extract_shotFhBh(x):
    if 'Forehand' in str(x):
        return 'Forehand'
    elif 'Backhand' in str(x):
        return 'Backhand'
    else:
        return None

def extract_shotDirection(x):
    if 'Crosscourt' in str(x):
        return 'Crosscourt'
    elif 'Down the Line' in str(x):
        return 'Down the Line'
    else:
        return None

# Apply the functions to create new columns, but only when shotFhBh and shotDirection are empty
shot_data['shotFhBh'] = shot_data['shotFhBh'].fillna(shot_data['shotType'].apply(extract_shotFhBh))
shot_data['shotDirection'] = shot_data['shotDirection'].fillna(shot_data['shotType'].apply(extract_shotDirection))

# Similarly, apply the functions to returnData, but only when shotFhBh and shotDirection are still empty
shot_data.loc[pd.notna(shot_data['returnData']) & pd.isna(shot_data['shotFhBh']), 'shotFhBh'] = shot_data.loc[pd.notna(shot_data['returnData']), 'returnData'].apply(extract_shotFhBh)
shot_data.loc[pd.notna(shot_data['returnData']) & pd.isna(shot_data['shotDirection']), 'shotDirection'] = shot_data.loc[pd.notna(shot_data['returnData']), 'returnData'].apply(extract_shotDirection)


# Display the resulting DataFrame
print(shot_data[['shotType', 'returnData', 'shotFhBh', 'shotDirection']].tail(15))

     shotType returnData  shotFhBh  shotDirection
964  Forehand        NaN  Forehand  Down the Line
965  Backhand        NaN  Backhand     Crosscourt
966  Backhand        NaN  Backhand     Crosscourt
967  Backhand        NaN  Backhand  Down the Line
968  Backhand        NaN  Backhand  Down the Line
969  Backhand        NaN  Backhand  Down the Line
970  Forehand        NaN  Forehand  Down the Line
971       NaN        NaN      None           None
972  Forehand        NaN  Forehand     Crosscourt
973       NaN        NaN      None           None
974  Backhand        NaN  Backhand     Crosscourt
975  Backhand        NaN  Backhand     Crosscourt
976  Backhand        NaN  Backhand     Crosscourt
977  Forehand        NaN  Forehand  Down the Line
978  Backhand        NaN  Backhand  Down the Line


- Fill in any NA for ['gameScore','setScore', 'serverName', 'player1Name', 'player2Name'] by forward filling
- Example: (1, NA, NA) -> (1, 1, 1)

# Add More Columns
- ReturnerName 
- shotHitBy
- isInsideOut
- isInsideIN


In [1078]:
# Get Returner Name for Shot DF

# Extract player1Name and player2Name from the first row of shot_data
first_row_shot_df = shot_data.iloc[0]
player1_name = first_row_shot_df['player1Name']
player2_name = first_row_shot_df['player2Name']

# # Can Also Manually inpute Player 1 and 2 name
# player1_name = "Kimmi Hance"
# player2_name = "Malaika Rapolu"

def get_returner_name(server_name):
    return player2_name if server_name == player1_name else player1_name

# Add 'Returner Name' column to point_df using the function
shot_data['returnerName'] = shot_data['serverName'].apply(get_returner_name)

print(f"Player 1 = {player1_name}, Player 2 = {player2_name}")
# print(shot_data[['serverName','returnerName']])

Player 1 = Kimmi Hance, Player 2 = Malaika Rapolu


In [1079]:
# Add Shot Hit By for each Shot

shot_data['shotHitBy'] = shot_data.apply(lambda row: row['serverName'] if row['shotInRally'] % 2 == 1 else row['returnerName'], axis=1)
# print(shot_data[['serverName', 'shotHitBy','pointNumber', 'shotInRally']].head(20))

# Right or Left Hand

In [1080]:
# Change this with whatever is player1Hand and player2Hand first Column Values
fill_in_player1hand = "Right"
fill_in_player2hand = "Right"

# Find the first non-null value in player1Hand column
player1hand = shot_data['player1Hand'].dropna().iloc[0] if 'player1Hand' in shot_data.columns else fill_in_player1hand
player2hand = shot_data['player2Hand'].dropna().iloc[0] if 'player2Hand' in shot_data.columns else fill_in_player2hand

# Display the updated values
print("player1Hand:", player1hand)
print("player2Hand:", player2hand)

shot_data['player1Hand'] = player1hand
shot_data['player2Hand'] = player2hand

player1Hand: Right
player2Hand: Right


In [1081]:
# Add columns for isInsideOut and isInsideIn, initially set to 0
shot_data['isInsideOut'] = None
shot_data['isInsideIn'] = None

# Iterate through rows
for index, row in shot_data.iterrows():
    shotHitBy = row['shotHitBy']
    player_hand = player1hand if shotHitBy == row['player1Name'] else player2hand
    
    if player_hand == "Right":
        if row['side'] == "Deuce" and row['shotType'] == "Backhand" and row['shotDirection'] == "Crosscourt":
            shot_data.at[index, 'isInsideOut'] = 1
        elif row['side'] == "Ad" and row['shotType'] == "Forehand" and row['shotDirection'] == "Crosscourt":
            shot_data.at[index, 'isInsideOut'] = 1
        elif row['side'] == "Deuce" and row['shotType'] == "Backhand" and row['shotDirection'] == "Down the Line":
            shot_data.at[index, 'isInsideIn'] = 1
        elif row['side'] == "Ad" and row['shotType'] == "Forehand" and row['shotDirection'] == "Down the Line":
            shot_data.at[index, 'isInsideIn'] = 1
    elif player_hand == "Left":
        if row['side'] == "Ad" and row['shotType'] == "Backhand" and row['shotDirection'] == "Crosscourt":
            shot_data.at[index, 'isInsideOut'] = 1
        elif row['side'] == "Deuce" and row['shotType'] == "Forehand" and row['shotDirection'] == "Crosscourt":
            shot_data.at[index, 'isInsideOut'] = 1
        elif row['side'] == "Ad" and row['shotType'] == "Backhand" and row['shotDirection'] == "Down the Line":
            shot_data.at[index, 'isInsideIn'] = 1
        elif row['side'] == "Deuce" and row['shotType'] == "Forehand" and row['shotDirection'] == "Down the Line":
            shot_data.at[index, 'isInsideIn'] = 1

In [1082]:
# Add the Ace column
shot_data['isAce'] = None

for index, row in shot_data.iterrows():
    if row['isPointEnd'] == 1:
        if row['shotInRally'] == 1: # last point is serve
            if (row['firstServeIn'] == 1 or row['secondServeIn'] == 1): # either first or second serve went in
                shot_data.at[index, 'isAce'] = 1

In [1083]:
# Add the DoubleFault column
shot_data['isDoubleFault'] = None

for index, row in shot_data.iterrows():
    if row['isPointEnd'] == 1:
        if row['shotInRally'] == 1: # last point is serve
            if (row['firstServeIn'] != 1 and row['secondServeIn'] != 1): # either first or second serve went in
                shot_data.at[index, 'isDoubleFault'] = 1

In [1084]:
# Add the 'pointWonBy' column
shot_data['pointWonBy'] = None

# Add the 'lastShotError' column
shot_data['lastShotError'] = 0

for index, row in shot_data.iterrows():
    if row['isPointEnd'] == 1:
        if row['shotInRally'] == 1: # last point is serve
            if row['isAce'] == 1: 
                shot_data.at[index, 'pointWonBy'] = row['serverName']
            elif row['isDoubleFault'] == 1: 
                shot_data.at[index, 'pointWonBy'] = row['returnerName']

                
        elif row['shotInRally'] != 1:
            if row['isErrorWideR'] == 1 or row['isErrorWideL'] == 1 or row['isErrorNet'] == 1 or row['isErrorLong'] == 1: # if error
                shot_data.at[index, 'lastShotError'] = 1
                
                if row['shotInRally'] % 2 == 0:
                    shot_data.at[index, 'pointWonBy'] = row['serverName']
                else:
                    shot_data.at[index, 'pointWonBy'] = row['returnerName']
        
            elif row['isWinner'] == 1:
                if row['shotInRally'] % 2 == 0:
                    shot_data.at[index, 'pointWonBy'] = row['returnerName']
                else:
                    shot_data.at[index, 'pointWonBy'] = row['serverName']


In [1085]:
shot_data['serverName'].unique()

array(['Malaika Rapolu', 'Kimmi Hance'], dtype=object)

In [1086]:
# Forward Fill GameScore, SetScore, tiebreakScore, serverName, player1Name, player2Name

columns_to_fill = ['gameScore','setScore', 'serverName', 'player1Name', 'player2Name', 'player1Hand', 'player2Hand'] 
# columns_to_fill = ['gameScore','setScore', 'tiebreakScore', 'serverName', 'player1Name', 'player2Name'] 
# Add player1Hand, player2Hand when it is in tagger

for column in columns_to_fill:
    shot_data[column].replace(['', 'na'], pd.NaT, inplace=True)
    shot_data[column] = shot_data[column].ffill()

In [1087]:
print(shot_data[(shot_data['isPointEnd'] == 1) & (shot_data['pointWonBy'].isnull())][['pointScore', 'gameScore','setScore', 'lastShotError', 'isWinner', 'isErrorWideR', 'isErrorWideL',
       'isErrorNet', 'isErrorLong', 'pointWonBy', 'serverName', 'shotInRally']])


    pointScore gameScore setScore  lastShotError  isWinner  isErrorWideR  \
375       0-40       4-5      0-0              0       NaN           NaN   
681      15-30       4-3      0-1              0       NaN           NaN   
894        NaN       6-6      0-1              0       NaN           NaN   
959        NaN       6-6      0-1              0       NaN           NaN   
978        NaN       6-6      0-1              0       NaN           NaN   

     isErrorWideL  isErrorNet  isErrorLong pointWonBy      serverName  \
375           NaN         NaN          NaN       None     Kimmi Hance   
681           NaN         NaN          NaN       None     Kimmi Hance   
894           NaN         NaN          NaN       None  Malaika Rapolu   
959           NaN         NaN          NaN       None     Kimmi Hance   
978           NaN         NaN          NaN       None     Kimmi Hance   

     shotInRally  
375            3  
681           14  
894           10  
959            5  
978      

In [1088]:


# Count where isPointStart = 1
count_isPointStart = (shot_data['isPointStart'] == 1).sum()

# Count where isPointEnd = 1
count_isPointEnd = (shot_data['isPointEnd'] == 1).sum()

print("Total count where isPointStart = 1:", count_isPointStart)
print("Total count where isPointEnd = 1:", count_isPointEnd)



# Initialize variables to keep track of the state
has_start = False
overlapping_ends = []

# Iterate through the DataFrame
for i in range(len(shot_data)):
    if shot_data['isPointStart'][i] == 1:
        has_start = True
    elif shot_data['isPointEnd'][i] == 1:
        if not has_start:
            overlapping_ends.append(i)
        else:
            has_start = False

# Display the rows where isPointEnd doesn't have a corresponding isPointStart
print("Rows where isPointEnd doesn't have a corresponding isPointStart:")
print(shot_data.iloc[overlapping_ends])

# Initialize variables to keep track of the state
start_indices = []
consecutive_starts = []

# Iterate through the DataFrame
for i in range(len(shot_data) - 1):  # Iterate until the second last row
    if shot_data['isPointStart'][i] == 1 and shot_data['isPointEnd'][i] != 1:
        # Check if the next row also has isPointStart without a corresponding isPointEnd
        if shot_data['isPointStart'][i + 1] == 1 and shot_data['isPointEnd'][i + 1] != 1:
            consecutive_starts.append(i + 1)

# Display the rows where isPointStart may double up without a corresponding isPointEnd
print("Rows where isPointStart may double up without a corresponding isPointEnd:")
print(shot_data.iloc[consecutive_starts])


Total count where isPointStart = 1: 143
Total count where isPointEnd = 1: 143
Rows where isPointEnd doesn't have a corresponding isPointStart:
Empty DataFrame
Columns: [pointScore, gameScore, setScore, isPointStart, pointStartTime, isPointEnd, pointEndTime, shotInRally, side, serverName, firstServeIn, firstServeZone, firstServeXCoord, firstServeYCoord, secondServeIn, secondServeZone, secondServeXCoord, secondServeYCoord, returnContactX, returnContactY, returnPlacementX, returnPlacementY, shotContactX, shotContactY, shotType, isVolley, isApproach, isDropshot, shotLocationX, shotLocationY, isWinner, isErrorWideR, isErrorWideL, isErrorNet, isErrorLong, pointNumber, clientTeam, Date, Division, Event, lineupPosition, player1Name, player2Name, Round, Surface, Notes, shotDirection, shotFhBh, tiebreakScore, returnData, returnerName, shotHitBy, player1Hand, player2Hand, isInsideOut, isInsideIn, isAce, isDoubleFault, pointWonBy, lastShotError]
Index: []

[0 rows x 60 columns]
Rows where isPointS

In [1089]:
player1Name = shot_data.iloc[0]['player1Name']

# Filter shot_data based on the conditions
approach_data_player1 = shot_data[(shot_data['isApproach'] == 1) & (shot_data['shotHitBy'] == player1Name)]

# Count the distinct pointNumbers
distinct_point_numbers = approach_data_player1['pointNumber'].nunique()

# Print the result
print("Number of distinct pointNumbers including isApproach = 1 and shotHitBy = player1Name:", distinct_point_numbers)

print(approach_data_player1)


Number of distinct pointNumbers including isApproach = 1 and shotHitBy = player1Name: 6
    pointScore gameScore setScore  isPointStart  pointStartTime  isPointEnd  \
79        0-40       1-0      0-0           NaN             NaN         NaN   
409      15-15       0-0      0-1           NaN             NaN         NaN   
543        0-0       2-2      0-1           NaN             NaN         NaN   
610        0-0       3-3      0-1           NaN             NaN         NaN   
761       15-0       5-5      0-1           NaN             NaN         1.0   
779       40-0       5-5      0-1           NaN             NaN         1.0   

     pointEndTime  shotInRally   side      serverName  ...    returnerName  \
79            NaN           13    NaN     Kimmi Hance  ...  Malaika Rapolu   
409           NaN            6  Deuce  Malaika Rapolu  ...     Kimmi Hance   
543           NaN           12     Ad  Malaika Rapolu  ...     Kimmi Hance   
610           NaN            4    NaN  Malaika

In [1090]:
shot_data.replace("", None, inplace=True)

# Output ShotCSV

In [1091]:
# # Ouput Improved Shot Csv HERE
# Assuming point_df is your DataFrame and player1Name and player2Name are the names from the first row
player1NameNoSpace = shot_data.iloc[0]['player1Name'].replace(" ", "")
player2NameNoSpace = shot_data.iloc[0]['player2Name'].replace(" ", "")

# Save DataFrame to CSV file with modified player names
shot_data.to_csv(f'{player1NameNoSpace}_{player2NameNoSpace}_Shot.csv', index=False)

In [1092]:
shot_data['pointNumber'].unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143])

# Below is for Point CSV

# Create Point DF

In [1093]:
# Creating point_df (with only 1 row for each pointNumber)
point_df = shot_data.drop_duplicates(subset='pointNumber')[['pointNumber']]

In [1094]:
point_df.shape

(143, 1)

In [1095]:
# Extract the first value of player1Name and player2Name from shot_data
first_player1Name = shot_data['player1Name'].iloc[0]
first_player2Name = shot_data['player2Name'].iloc[0]

# Fill in the first value into all rows of point_df['player1Name'] and point_df['player2Name']
point_df['player1Name'] = first_player1Name
point_df['player2Name'] = first_player2Name

# Add Scores


In [1096]:

point_df['pointScore'] = shot_data.groupby('pointNumber')['pointScore'].first().values
point_df['gameScore'] = shot_data.groupby('pointNumber')['gameScore'].first().values
point_df['tiebreakScore'] = shot_data.groupby('pointNumber')['tiebreakScore'].first().values

# Add Side

In [1097]:
# Group shot_data by 'pointNumber' and get the first 'side' value for each group
side_values = shot_data.groupby('pointNumber')['side'].first().reset_index()

point_df['side'] = side_values['side'].values

In [1098]:
# Adds Server and Returner Names and pointScore

point_df['serverName'] = shot_data.groupby('pointNumber')['serverName'].first().values
point_df['returnerName'] = shot_data.groupby('pointNumber')['returnerName'].first().values


In [1099]:
# Add Start and End times per point

for index, row in shot_data.iterrows():
    point_number = row['pointNumber']
    
    if row['isPointStart'] == 1:
        point_df.loc[point_df['pointNumber'] == point_number, 'Position'] = row['pointStartTime']
    if row['isPointEnd'] == 1:
        point_df.loc[point_df['pointNumber'] == point_number, 'pointEndPosition'] = row['pointEndTime']

# Add Duration
point_df['Duration'] = point_df['pointEndPosition'] - point_df['Position']

# Add Rally Column
- rallyCount (x amount)
- rallyCountFreq (1-4,5-8,9-12,13+)

In [1100]:
# Find the highest shotInRally for each pointNumber in shot_data
max_rally_per_point = shot_data.groupby('pointNumber')['shotInRally'].max().reset_index()
point_df['rallyCount'] = list(max_rally_per_point['shotInRally'])

# Add 'rallyCountFreq' column based on specified conditions
point_df['rallyCountFreq'] = point_df['rallyCount'].apply(lambda x: '1 - 4' if 1 <= x <= 4 else ('5 - 8' if 5 <= x <= 8 else ('9 - 12' if 9 <= x <= 12 else ('13 +' if x >= 13 else 'Error'))))


# Serve 
- firstServeIn
- secondServeIn
- serveResult
- serveInPlacement

In [1101]:
# Add firstServeIn and secondServeIn

# Add firstServeIn and secondServeIn columns
point_df['firstServeIn'] = 0
point_df['secondServeIn'] = 0

for point_number in shot_data['pointNumber'].unique():
    # Check if firstServeIn is 1 for the given pointNumber in shot_data
    if any((shot_data['pointNumber'] == point_number) & (shot_data['firstServeIn'] == 1)):
        point_df.loc[point_df['pointNumber'] == point_number, 'firstServeIn'] = 1
    
    # Check if secondServeIn is 1 for the given pointNumber in shot_data
    if any((shot_data['pointNumber'] == point_number) & (shot_data['secondServeIn'] == 1)):
        point_df.loc[point_df['pointNumber'] == point_number, 'secondServeIn'] = 1

In [1169]:
# Add serveResult and serveInPlacement

point_df['serveResult'] = None
point_df['serveInPlacement'] = None

for point_number in shot_data['pointNumber'].unique():
    if any((shot_data['pointNumber'] == point_number) & (shot_data['firstServeIn'] == 1)):
        point_df.loc[point_df['pointNumber'] == point_number, 'serveResult'] = '1st Serve In'
        point_df.loc[point_df['pointNumber'] == point_number, 'serveInPlacement'] = shot_data['firstServeZone']
    elif any((shot_data['pointNumber'] == point_number) & (shot_data['secondServeIn'] == 1)):
        point_df.loc[point_df['pointNumber'] == point_number, 'serveResult'] = '2nd Serve In'
        point_df.loc[point_df['pointNumber'] == point_number, 'serveInPlacement'] = shot_data['secondServeZone']    
    else: 
        point_df.loc[point_df['pointNumber'] == point_number, 'serveResult'] = 'Double Fault'
        point_df.loc[point_df['pointNumber'] == point_number, 'serveInPlacement'] = np.nan
        
    point_df.loc[point_df['pointNumber'] == point_number, 'firstServeZone'] = shot_data['firstServeZone']
    point_df.loc[point_df['pointNumber'] == point_number, 'secondServeZone'] = shot_data['secondServeZone']
    
# Display the result
print(point_df['serveInPlacement'].unique())




['Body' 'Wide' 'T' nan]


In [1103]:
serve_zones = shot_data.loc[shot_data['shotInRally'] == 1, ['pointNumber', 'firstServeZone', 'secondServeZone', 'firstServeIn', 'secondServeIn']].drop_duplicates()


point_df['firstServeZone'] = shot_data.groupby('pointNumber')['firstServeZone'].first().values
point_df['secondServeZone'] = shot_data.groupby('pointNumber')['secondServeZone'].first().values



In [1104]:
double_fault_rows = point_df[point_df['serveResult'] == 'Double Fault']
# print(double_fault_rows)

# Ace

In [1105]:
point_df['isAce'] = ((point_df['rallyCount'] == 1) & ((point_df['serveResult'] != "Double Fault")))

# Display the resulting DataFrame
print(point_df[['pointNumber', 'rallyCount', 'isAce']].head(14))

    pointNumber  rallyCount  isAce
0             1           8  False
8             2           2  False
10            3           3  False
13            4          11  False
24            5          14  False
38            6           4  False
42            7           7  False
49            8           3  False
52            9          11  False
63           10           3  False
66           11           1   True
67           12          15  False
82           13           1  False
83           14          12  False


# Return

In [1106]:
# Assuming point_df is your DataFrame

# Set the initial values of 'returnDirection' and 'returnHand' columns to None
point_df['returnDirection'] = None
point_df['returnFhBh'] = None

# Iterate through pointNumber in shot_data
for point_number in shot_data['pointNumber'].unique():
    # Check if shotInRally == 2 exists for the given pointNumber
    if 2 in shot_data.loc[shot_data['pointNumber'] == point_number, 'shotInRally'].values:
        # Get the information from the corresponding row
        row_with_return_info = shot_data[(shot_data['pointNumber'] == point_number) & (shot_data['shotInRally'] == 2)].iloc[0]

        # Assign values to 'returnDirection' and 'returnHand' columns
        point_df.loc[point_df['pointNumber'] == point_number, 'returnDirection'] = row_with_return_info['shotDirection']
        point_df.loc[point_df['pointNumber'] == point_number, 'returnFhBh'] = row_with_return_info['shotFhBh']

# Display the modified DataFrame
print(point_df.head(10))


    pointNumber  player1Name     player2Name pointScore gameScore  \
0             1  Kimmi Hance  Malaika Rapolu        0-0       0-0   
8             2  Kimmi Hance  Malaika Rapolu       0-15       0-0   
10            3  Kimmi Hance  Malaika Rapolu      15-15       0-0   
13            4  Kimmi Hance  Malaika Rapolu      15-30       0-0   
24            5  Kimmi Hance  Malaika Rapolu      15-40       0-0   
38            6  Kimmi Hance  Malaika Rapolu      30-40       0-0   
42            7  Kimmi Hance  Malaika Rapolu      40-40       0-0   
49            8  Kimmi Hance  Malaika Rapolu        0-0       1-0   
52            9  Kimmi Hance  Malaika Rapolu       0-15       1-0   
63           10  Kimmi Hance  Malaika Rapolu       0-30       1-0   

   tiebreakScore   side      serverName    returnerName  Position  ...  \
0           None  Deuce  Malaika Rapolu     Kimmi Hance    9486.0  ...   
8           None     Ad  Malaika Rapolu     Kimmi Hance   75253.0  ...   
10          None  

# Add Error Column

In [1107]:
# Create an empty DataFrame to store the results
error_results = pd.DataFrame(columns=['errorType', 'pointNumber'])

# Iterate through entire shot_data
for index, row in shot_data.iterrows():
    pointNumber = row['pointNumber']
    point_error_value = None
    
    if row['isErrorWideR'] == 1:
        point_error_value = 'Wide Right'
    elif row['isErrorWideL'] == 1:
        point_error_value = 'Wide Left'
    elif 'isErrorNet' in row and row['isErrorNet'] == 1:
        point_error_value = 'Net'
    elif row['isErrorLong'] == 1:
        point_error_value = 'Long'
    

    # If an error is found, append the result to the error_results DataFrame
    if point_error_value is not None:
        error_results = pd.concat([error_results, pd.DataFrame({'pointNumber': [pointNumber], 'errorType': [point_error_value]})], ignore_index=True)


# Drop duplicates based on 'pointNumber'
error_results = error_results.drop_duplicates(subset=['pointNumber'])

In [1108]:
# Create a dictionary mapping 'pointNumber' to 'errorType' in error_results
error_type_mapping = dict(zip(error_results['pointNumber'], error_results['errorType']))

# Create 'errorType' column in point_df based on the mapping
point_df['errorType'] = point_df['pointNumber'].map(error_type_mapping)

point_df = point_df.replace({np.nan: None})

# Return Stuff

In [1109]:
def get_return_error(row):
    if row['rallyCount'] == 2:
        return row['errorType']
    else:
        return None

point_df.loc[point_df['pointNumber'] == point_number, 'serveInPlacement'] = shot_data['secondServeZone']  

# Apply the functions to create the new columns
point_df['returnError'] = point_df.apply(get_return_error, axis=1)


In [1110]:
# print(point_df["returnError"].unique())
# print(point_df[['rallyCount','returnDirection','returnFhBh']])

# Last Shot

In [1111]:

point_df['lastShotDirection'] = None
point_df['lastShotFhBh'] = None
point_df['lastShotHitBy'] = None  
point_df['lastShotResult'] = None  

# Iterate through unique pointNumbers in shot_data
for point_number in shot_data['pointNumber'].unique():
    # Check if isPointEnd == 1 exists for the given pointNumber
    if 1 in shot_data.loc[shot_data['pointNumber'] == point_number, 'isPointEnd'].values:
        # Get the information from the corresponding row
        row_with_lastshot_info = shot_data[(shot_data['pointNumber'] == point_number) & (shot_data['isPointEnd'] == 1)].iloc[0]

        # Assign values to 'lastShotDirection' and 'lastShotFhBh' columns
        point_df.loc[point_df['pointNumber'] == point_number, 'lastShotDirection'] = row_with_lastshot_info['shotDirection']
        point_df.loc[point_df['pointNumber'] == point_number, 'lastShotFhBh'] = row_with_lastshot_info['shotFhBh']
        point_df.loc[point_df['pointNumber'] == point_number, 'lastShotHitBy'] = row_with_lastshot_info['shotHitBy']
        
        # Determine lastShotResult based on conditions
        if row_with_lastshot_info['isWinner'] == 1:
            point_df.loc[point_df['pointNumber'] == point_number, 'lastShotResult'] = "Winner"
        elif row_with_lastshot_info['lastShotError'] == 1:
            point_df.loc[point_df['pointNumber'] == point_number, 'lastShotResult'] = "Error"

In [1112]:
# # Display the modified DataFrame
# print(point_df[['rallyCount','lastShotDirection','lastShotFhBh', 'lastShotHitBy']])

In [1113]:
print(point_df.columns)

Index(['pointNumber', 'player1Name', 'player2Name', 'pointScore', 'gameScore',
       'tiebreakScore', 'side', 'serverName', 'returnerName', 'Position',
       'pointEndPosition', 'Duration', 'rallyCount', 'rallyCountFreq',
       'firstServeIn', 'secondServeIn', 'serveResult', 'serveInPlacement',
       'firstServeZone', 'secondServeZone', 'isAce', 'returnDirection',
       'returnFhBh', 'errorType', 'returnError', 'lastShotDirection',
       'lastShotFhBh', 'lastShotHitBy', 'lastShotResult'],
      dtype='object')


# pointWonBy

In [1114]:
# Initialize variables to keep track of the state
prev_point_number = None
point_won_by_list = []

# Iterate through the DataFrame
for index, row in shot_data.iterrows():
    if row['isPointEnd'] == 1:
        # Check if pointNumber is different and consecutively increasing
        if prev_point_number is None or row['pointNumber'] == prev_point_number + 1:
            # Append pointWonBy to the list
            point_won_by_list.append(row['pointWonBy'])
            prev_point_number = row['pointNumber']
        else:
            print("Error: Point numbers are not different or consecutively increasing.")
            break

# Add point_won_by_list as a new column to point_df
point_df['pointWonBy'] = point_won_by_list

# Print point_df
print(point_df)


print(point_df['player1Name'].unique())
print(point_df['player2Name'].unique())
print(point_df['serverName'].unique())
print(point_df['returnerName'].unique())

# # Filter shot_data for rows where isPointEnd is 1
# filtered_df = shot_data[shot_data['isPointEnd'] == 1]

# # Record values for specified columns
# pointWonBy_df2 = filtered_df[['player1Name', 'player2Name', 'serverName', 'isWinner', 'pointNumber', 'shotInRally']].copy()

# # Determine returnerName based on serverName
# pointWonBy_df2['returnerName'] = pointWonBy_df2.apply(lambda row: row['player2Name'] if row['serverName'] == row['player1Name'] else row['player1Name'], axis=1)


# for index, row in pointWonBy_df2.iterrows():
#     if (row['isWinner'] == 1 and row['shotInRally'] % 2 == 1) or (row['isWinner'] != 1 and row['shotInRally'] % 2 == 0):
#         pointWonBy_df2.at[index, 'pointWonBy'] = row['serverName']
#     elif (row['isWinner'] == 1 and row['shotInRally'] % 2 == 0) or (row['isWinner'] != 1 and row['shotInRally'] % 2 == 1):
#         pointWonBy_df2.at[index, 'pointWonBy'] = row['returnerName']
#     else:
#         pointWonBy_df2.at[index, 'pointWonBy'] = None

# point_df['pointWonBy'] = pointWonBy_df2['pointWonBy'].values

     pointNumber  player1Name     player2Name pointScore gameScore  \
0              1  Kimmi Hance  Malaika Rapolu        0-0       0-0   
8              2  Kimmi Hance  Malaika Rapolu       0-15       0-0   
10             3  Kimmi Hance  Malaika Rapolu      15-15       0-0   
13             4  Kimmi Hance  Malaika Rapolu      15-30       0-0   
24             5  Kimmi Hance  Malaika Rapolu      15-40       0-0   
..           ...          ...             ...        ...       ...   
951          139  Kimmi Hance  Malaika Rapolu       None       6-6   
955          140  Kimmi Hance  Malaika Rapolu       None       6-6   
960          141  Kimmi Hance  Malaika Rapolu       None       6-6   
971          142  Kimmi Hance  Malaika Rapolu       None       6-6   
973          143  Kimmi Hance  Malaika Rapolu       None       6-6   

    tiebreakScore   side      serverName    returnerName   Position  ...  \
0            None  Deuce  Malaika Rapolu     Kimmi Hance     9486.0  ...   
8      

In [1115]:
# # Display the resulting DataFrame
print(point_df['pointWonBy'].unique())

['Kimmi Hance' 'Malaika Rapolu' None]


# Break Point

In [1116]:
# List of values to check for in 'Name' column
break_point_values = ['0-40', '15-40', '30-40', '40-40 (Deuce)', '40-40 (Ad)']

# Create 'isBreakPoint' column in point_df
point_df['isBreakPoint'] = point_df['pointScore'].isin(break_point_values)

In [1117]:
# Display the resulting DataFrame
# print(point_df[['pointNumber', 'Name', 'isBreakPoint']])

# Add Side

In [1118]:
# # List of values to check for in 'Name' column for Deuce
# deuce_values = ['0-0' ,'15-15','40-15','30-0' , '0-30','15-40' ,'40-40 (Deuce)','30-30']

# # Create 'servingSide' column in point_df
# point_df['servingSide'] = np.where(point_df['Name'].isin(deuce_values), 'Deuce', 'Ad')

In [1119]:
# # Display the resulting DataFrame
# print(point_df[['pointNumber', 'Name', 'servingSide']])

# At Net

- replace later with atNetAutomatically value

In [1120]:
import numpy as np

# Copy the DataFrame
at_net_df2 = shot_data.copy()

# Record values for specified columns
at_net_df2 = at_net_df2[['player1Name', 'player2Name', 'isVolley', 'serverName', 'shotHitBy', 'returnerName', 'pointNumber', 'shotInRally']].copy()

# Add columns for aggregation
at_net_df2['atNetPlayer1'] = np.where((at_net_df2['isVolley'] == 1) & (at_net_df2['shotHitBy'] == at_net_df2['player1Name']), 1, 0)
at_net_df2['atNetPlayer2'] = np.where((at_net_df2['isVolley'] == 1) & (at_net_df2['shotHitBy'] == at_net_df2['player2Name']), 1, 0)

# Aggregate atNetPlayer1 and atNetPlayer2 based on pointNumber
at_net_df2['atNetPlayer1_Agg'] = at_net_df2.groupby('pointNumber')['atNetPlayer1'].transform('max')
at_net_df2['atNetPlayer2_Agg'] = at_net_df2.groupby('pointNumber')['atNetPlayer2'].transform('max')

# Drop the duplicate rows created during aggregation
at_net_df2.drop_duplicates(subset=['pointNumber'], inplace=True)

point_df['atNetPlayer1'] = at_net_df2['atNetPlayer1_Agg'].values
point_df['atNetPlayer2'] = at_net_df2['atNetPlayer2_Agg'].values

In [1121]:
print(point_df.tail(10))

     pointNumber  player1Name     player2Name pointScore gameScore  \
911          134  Kimmi Hance  Malaika Rapolu       None       6-6   
924          135  Kimmi Hance  Malaika Rapolu       None       6-6   
930          136  Kimmi Hance  Malaika Rapolu       None       6-6   
940          137  Kimmi Hance  Malaika Rapolu       None       6-6   
950          138  Kimmi Hance  Malaika Rapolu       None       6-6   
951          139  Kimmi Hance  Malaika Rapolu       None       6-6   
955          140  Kimmi Hance  Malaika Rapolu       None       6-6   
960          141  Kimmi Hance  Malaika Rapolu       None       6-6   
971          142  Kimmi Hance  Malaika Rapolu       None       6-6   
973          143  Kimmi Hance  Malaika Rapolu       None       6-6   

    tiebreakScore side      serverName    returnerName   Position  ...  \
911           4-3   Ad     Kimmi Hance  Malaika Rapolu  7252322.0  ...   
924           4-4   Ad     Kimmi Hance  Malaika Rapolu  7297998.0  ...   
930    

# Set Score
- Add Set Score
- Add setNum

In [1122]:
# Group shot_data by 'pointNumber' and get the first value of 'setScore'
setScore_df = shot_data.groupby('pointNumber')['setScore'].first().reset_index()

point_df['setScore'] = setScore_df['setScore'].values

In [1123]:
# Extract numbers from 'setScore' and calculate the sum plus 1
point_df['setNum'] = point_df['setScore'].apply(lambda x: sum(int(char) for char in x if char.isdigit()) + 1)

In [1124]:
# Create an empty DataFrame to store gameScore and setScore
setScore = shot_data.groupby('pointNumber').first()['setScore'].reset_index()
gameScore = shot_data.groupby('pointNumber').first()['gameScore'].reset_index()
pointScore = shot_data.groupby('pointNumber').first()['pointScore'].reset_index()
tiebreakScore = shot_data.groupby('pointNumber').first()['tiebreakScore'].reset_index()


point_df['setScore'] = setScore['setScore'].values
point_df['gameScore'] = gameScore['gameScore'].values
point_df['pointScore'] = pointScore['pointScore'].values
point_df['tiebreakScore'] = tiebreakScore['tiebreakScore'].values



In [1125]:
# Add Game Number, Set Number, Game/Set/Point for each player

point_df[['player1SetScore', 'player2SetScore']] = point_df['setScore'].str.split('-', expand=True)
point_df[['player1GameScore', 'player2GameScore']] = point_df['gameScore'].str.split('-', expand=True)
point_df[['player1PointScore', 'player2PointScore']] = point_df['pointScore'].str.split('-', expand=True)
point_df[['player1TiebreakScore', 'player2TiebreakScore']] = point_df['tiebreakScore'].str.split('-', expand=True)


def calculate_game_number(score):
    return int(score.split('-')[0]) + int(score.split('-')[1])+1

# Apply the function to create the 'gameNumber' column
point_df['gameNumber'] = point_df['gameScore'].apply(calculate_game_number)

In [1126]:
print(point_df['gameNumber'])

0       1
8       1
10      1
13      1
24      1
       ..
951    13
955    13
960    13
971    13
973    13
Name: gameNumber, Length: 143, dtype: int64


In [1127]:
print(point_df['serveResult'])

0      2nd Serve In
8      1st Serve In
10     2nd Serve In
13     2nd Serve In
24     2nd Serve In
           ...     
951    1st Serve In
955    2nd Serve In
960    1st Serve In
971    2nd Serve In
973    1st Serve In
Name: serveResult, Length: 143, dtype: object


In [1128]:
# Add the 'player1ServeResult' column
point_df['player1ServeResult'] = None

# Set player1ServeResult based on conditions
point_df.loc[point_df['serverName'] == point_df['player1Name'], 'player1ServeResult'] = point_df['serveResult']

In [1129]:
point_df.loc[point_df['isAce'] == True, 'player1ServeResult'] = 'Ace'


In [1130]:
print(point_df['serveInPlacement'].unique())

['Body' 'Wide' 'T' None nan]


In [1131]:
# Add the 'player1ServePlacement' column
point_df['player1ServePlacement'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['serverName'] == point_df['player1Name'], 'player1ServePlacement'] = point_df['side'] + ': ' + point_df['serveInPlacement']

In [1132]:
# Add the 'player1ReturnPlacement' column
point_df['player1ReturnPlacement'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['returnerName'] == point_df['player1Name'], 'player1ReturnPlacement'] = point_df['returnDirection']

In [1133]:
# Add the 'player1ReturnFhBh' column
point_df['player1ReturnFhBh'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['returnerName'] == point_df['player1Name'], 'player1ReturnFhBh'] = point_df['returnFhBh']

In [1134]:
# Add the 'player1LastShotFhBh' column
point_df['player1LastShotPlacement'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['lastShotHitBy'] == point_df['player1Name'], 'player1LastShotPlacement'] = point_df['lastShotDirection']

In [1135]:
# Add the 'player1LastShotFhBh' column
point_df['player1LastShotFhBh'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['lastShotHitBy'] == point_df['player1Name'], 'player1LastShotFhBh'] = point_df['lastShotFhBh']

In [1136]:
# Add the 'player1LastShotResult' column
point_df['player1LastShotResult'] = None

# Set player1LastShotResult based on conditions, excluding 'Ace' and 'Double Fault'
point_df.loc[
    (point_df['lastShotHitBy'] == point_df['player1Name']) & 
    ~point_df['player1ServeResult'].isin(['Ace', 'Double Fault']), 
    'player1LastShotResult'
] = point_df['lastShotResult']


# Add Name
- Changes PointScore
- Adds pointScoreCopy

In [1137]:
print(point_df[['setNum', 'gameScore','pointScore','tiebreakScore']])

     setNum gameScore pointScore tiebreakScore
0         1       0-0        0-0          None
8         1       0-0       0-15          None
10        1       0-0      15-15          None
13        1       0-0      15-30          None
24        1       0-0      15-40          None
..      ...       ...        ...           ...
951       2       6-6       None           6-6
955       2       6-6       None           6-7
960       2       6-6       None           7-7
971       2       6-6       None           8-8
973       2       6-6       None           8-9

[143 rows x 4 columns]


In [1138]:
# Change pointScore to the specified format
point_df['Name'] = point_df.apply(lambda row: f"Set {row['setNum']}: {row['gameScore']}, {row['tiebreakScore']} {row['serverName']} Serving" if pd.notna(row['tiebreakScore']) else f"Set {row['setNum']}: {row['gameScore']}, {row['pointScore']} {row['serverName']} Serving", axis=1)


In [1139]:
print(point_df[['setNum', 'gameScore','pointScore','tiebreakScore','Name']])

     setNum gameScore pointScore tiebreakScore  \
0         1       0-0        0-0          None   
8         1       0-0       0-15          None   
10        1       0-0      15-15          None   
13        1       0-0      15-30          None   
24        1       0-0      15-40          None   
..      ...       ...        ...           ...   
951       2       6-6       None           6-6   
955       2       6-6       None           6-7   
960       2       6-6       None           7-7   
971       2       6-6       None           8-8   
973       2       6-6       None           8-9   

                                         Name  
0      Set 1: 0-0, 0-0 Malaika Rapolu Serving  
8     Set 1: 0-0, 0-15 Malaika Rapolu Serving  
10   Set 1: 0-0, 15-15 Malaika Rapolu Serving  
13   Set 1: 0-0, 15-30 Malaika Rapolu Serving  
24   Set 1: 0-0, 15-40 Malaika Rapolu Serving  
..                                        ...  
951       Set 2: 6-6, 6-6 Kimmi Hance Serving  
955       Set 2

# EDA

In [1140]:
# Can input CSV Directly here for statistics functions

# your_file_name = "filename.csv"
# point_df = pd.read_csv(your_file_name)

In [1141]:
first_player1Name = point_df['player1Name'].iloc[0]



# Display the results
print(f"\nServe Results for {first_player1Name}:")

# Assuming point_df is your DataFrame
total_serves = len(point_df[point_df['serverName'] == first_player1Name])
first_serve_in_count = len(point_df[(point_df['serverName'] == first_player1Name) & (point_df['firstServeIn'] == 1)])
first_serve_won_count = len(point_df[(point_df['serverName'] == first_player1Name) & (point_df['firstServeIn'] == 1) & (point_df['pointWonBy'] == first_player1Name)])
percentage_first_serve_in = (first_serve_in_count / total_serves) * 100 if total_count > 0 else 0
percentage_first_serve_won = (first_serve_won_count / first_serve_in_count) * 100 if first_serve_in_count > 0 else 0

second_serve_total_count = len(point_df[(point_df['serverName'] == first_player1Name) & (point_df['firstServeIn'] == 0)])
second_serve_in_count = len(point_df[(point_df['serverName'] == first_player1Name) & (point_df['firstServeIn'] == 0)& (point_df['secondServeIn'] == 1)])
second_serve_won_count = len(point_df[(point_df['serverName'] == first_player1Name) & (point_df['firstServeIn'] == 0)& (point_df['secondServeIn'] == 1) & (point_df['pointWonBy'] == first_player1Name)])
percentage_second_serve_in = (second_serve_in_count / second_serve_total_count) * 100 if second_serve_total_count > 0 else 0
percentage_second_serve_won = (second_serve_won_count / second_serve_in_count) * 100 if second_serve_in_count > 0 else 0



# Display the results
print("\nTotal Serves:", total_serves)
print("First Serve In (Count):", first_serve_in_count)
print("First Serve Won (Count):", first_serve_won_count)
print(f"First Serve In (%): {percentage_first_serve_in:.2f}%")
print(f"First Serve Won (%): {percentage_first_serve_won:.2f}%")

print("Second Serve In (Count):", second_serve_in_count)
print("Second Serve Total (Count):", second_serve_total_count)
print("Second Serve Won (Count):", second_serve_won_count)
print(f"Second Serve In (%): {percentage_second_serve_in:.2f}%")
print(f"Second Serve Won (%): {percentage_second_serve_won:.2f}%")

# Assuming point_df is your DataFrame
count_is_ace = (point_df[point_df['serverName'] == first_player1Name]['isAce']).sum()
count_is_double_fault = ((point_df['serverName'] == first_player1Name) & (point_df['serveResult'] == "Double Fault")).sum()

# Display the results
print("Ace (Count):", count_is_ace)
print("Double Fault (Count):", count_is_double_fault)

# Count of rows where serverName is equal to the first row of player1Name and pointWonBy is equal to the first row of player1Name
total_service_points_won = len(point_df[(point_df['serverName'] == first_player1Name) & (point_df['pointWonBy'] == first_player1Name)])
total_service_points_won_percentage = total_service_points_won / total_serves *100

# Display the results
print(f"Points Won on Serve (Count) {total_service_points_won}")

print(f"Points Won on Serve (%): {total_service_points_won_percentage:.2f}%")

# Assuming point_df is your DataFrame
return_points = point_df[(point_df['returnerName'] == first_player1Name) & (point_df['rallyCount'] >= 2)]

total_return = len(return_points)
returnMade = len(return_points[(return_points['rallyCount'] > 2) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error'))])
returnError = len(return_points[(return_points['lastShotResult'] == 'Error') & (return_points['rallyCount'] == 2)])
returnWinner = len(return_points[(return_points['lastShotResult'] == 'Winner') & (return_points['rallyCount'] == 2)])
returnMadePercentage = returnMade/total_return

returnWonByPlayer1 = len(return_points[return_points['pointWonBy'] == first_player1Name])
returnWonByPlayer1Percentage = returnWonByPlayer1 / total_return * 100 if total_return > 0 else 0

deuceReturnCount = len(return_points[return_points['side'] == 'Deuce'])
adReturnCount = len(return_points[return_points['side'] == 'Ad'])


deuceReturnMade = len(return_points[(return_points['side'] == 'Deuce') & ((return_points['rallyCount'] > 2) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error')))])
adReturnMade = len(return_points[(return_points['side'] == 'Ad') & ((return_points['rallyCount'] > 2) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error')))])

deuceReturnMadePercentage = deuceReturnMade/deuceReturnCount
adReturnMadePercentage = adReturnMade/adReturnCount

deuceReturnWonByPlayer1 = len(return_points[(return_points['side'] == 'Deuce') & (return_points['pointWonBy'] == first_player1Name) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error'))])
adReturnWonByPlayer1 = len(return_points[(return_points['side'] == 'Ad') & (return_points['pointWonBy'] == first_player1Name) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error'))])

deuceReturnWonByPlayer1Percentage = deuceReturnWonByPlayer1 / deuceReturnCount * 100 if deuceReturnCount > 0 else 0
adReturnWonByPlayer1Percentage = adReturnWonByPlayer1 / adReturnCount * 100 if adReturnCount > 0 else 0




print(f"\nReturn Results for {first_player1Name}:\n")

print("Total Return (Count):", total_return)
print("Return Won (Count):", returnWonByPlayer1)
print("Return Won (%):", returnWonByPlayer1Percentage)

print("\nReturn Made (Count):", returnMade)
print("Return Made (%):", returnMadePercentage)
print("Return Error (Count):", returnError)
print("Return Winner (Count):", returnWinner)

print("\nDeuce Return (Count):", deuceReturnCount)
print("Deuce Return Made (Count):", deuceReturnMade)
print("Deuce Return Made (%):", deuceReturnMadePercentage)
print("Deuce Return Won by Player1 (%):", deuceReturnWonByPlayer1Percentage)
print("Deuce Return Won by Player1 (Count):", deuceReturnWonByPlayer1)


print("\nAd Return (Count):", adReturnCount)
print("Ad Return Made (Count):", adReturnMade)
print("Ad Return Made (%):", adReturnMadePercentage)
print("Ad Return Won by Player1 (Count):", adReturnWonByPlayer1)
print("Ad Return Won by Player1 (%):", adReturnWonByPlayer1Percentage)

# Assuming return_points is your DataFrame
deuce_return_points = return_points[(return_points['side'] == 'Deuce') & (return_points['returnerName'] == first_player1Name) & (return_points['rallyCount'] >= 2)]

# Deuce Return Points Separated by returnFhBh
deuce_forehand_return_points = deuce_return_points[deuce_return_points['returnFhBh'] == 'Forehand']
deuce_backhand_return_points = deuce_return_points[deuce_return_points['returnFhBh'] == 'Backhand']


# Count for Deuce Return Points - Made
count_deuce_forehand_made = len(deuce_forehand_return_points[(deuce_forehand_return_points['rallyCount'] > 2) | ((deuce_forehand_return_points['rallyCount'] == 2) & (deuce_forehand_return_points['lastShotResult'] != 'Error'))])
count_deuce_backhand_made = len(deuce_backhand_return_points[(deuce_backhand_return_points['rallyCount'] > 2) | ((deuce_backhand_return_points['rallyCount'] == 2) & (deuce_backhand_return_points['lastShotResult'] != 'Error'))])

# Count for Deuce Return Points - Error
count_deuce_forehand_error = len(deuce_forehand_return_points[(deuce_forehand_return_points['lastShotResult'] == 'Error') & (deuce_forehand_return_points['rallyCount'] == 2)])
count_deuce_backhand_error = len(deuce_backhand_return_points[(deuce_backhand_return_points['lastShotResult'] == 'Error') & (deuce_backhand_return_points['rallyCount'] == 2)])

# Display the counts
print("Deuce Forehand Return Points - Made:", count_deuce_forehand_made)
print("Deuce Forehand Return Points - Error:", count_deuce_forehand_error)

print("\nDeuce Backhand Return Points - Made:", count_deuce_backhand_made)
print("Deuce Backhand Return Points - Error:", count_deuce_backhand_error)

# Assuming return_points is your DataFrame
ad_return_points = return_points[(return_points['side'] == 'Ad') & (return_points['returnerName'] == first_player1Name) & (return_points['rallyCount'] >= 2)]

# Ad Return Points Separated by returnFhBh
ad_forehand_return_points = ad_return_points[ad_return_points['returnFhBh'] == 'Forehand']
ad_backhand_return_points = ad_return_points[ad_return_points['returnFhBh'] == 'Backhand']

# Count for Ad Return Points - Made
count_ad_forehand_made = len(ad_forehand_return_points[(ad_forehand_return_points['rallyCount'] > 2) | ((ad_forehand_return_points['rallyCount'] == 2) & (ad_forehand_return_points['lastShotResult'] != 'Error'))])
count_ad_backhand_made = len(ad_backhand_return_points[(ad_backhand_return_points['rallyCount'] > 2) | ((ad_backhand_return_points['rallyCount'] == 2) & (ad_backhand_return_points['lastShotResult'] != 'Error'))])

# Count for Ad Return Points - Error
count_ad_forehand_error = len(ad_forehand_return_points[(ad_forehand_return_points['lastShotResult'] == 'Error') & (ad_forehand_return_points['rallyCount'] == 2)])
count_ad_backhand_error = len(ad_backhand_return_points[(ad_backhand_return_points['lastShotResult'] == 'Error') & (ad_backhand_return_points['rallyCount'] == 2)])

# Display the counts
print("Ad Forehand Return Points - Made:", count_ad_forehand_made)
print("Ad Forehand Return Points - Error:", count_ad_forehand_error)

print("\nAd Backhand Return Points - Made:", count_ad_backhand_made)
print("Ad Backhand Return Points - Error:", count_ad_backhand_error)

print(f"\nAt Net Results for {first_player1Name}:\n")


# Total points where atNetPlayer1 = 1
total_at_net_player1 = len(point_df[point_df['atNetPlayer1'] == 1])

# Percentage of points where atNetPlayer1 = 1 out of total points
percentage_at_net_player1 = (total_at_net_player1 / len(point_df)) * 100 if len(point_df) > 0 else 0

# Display the total count and percentage of points where atNetPlayer1 = 1
print(f"Total Net Points for {first_player1Name}: {total_at_net_player1}")
print(f"Percentage of Net Points for {first_player1Name}: {percentage_at_net_player1:.2f}%")

# Points where atNetPlayer1 = 1 and pointWonBy = first_player1Name
at_net_player1_and_won_by_player1 = len(point_df[(point_df['atNetPlayer1'] == 1) & (point_df['pointWonBy'] == first_player1Name)])

# Percentage of points where atNetPlayer1 = 1 and pointWonBy = first_player1Name out of total points where atNetPlayer1 = 1
percentage_at_net_player1_and_won_by_player1 = (at_net_player1_and_won_by_player1 / total_at_net_player1) * 100 if total_at_net_player1 > 0 else 0

# Display the count and percentage of points where atNetPlayer1 = 1 and pointWonBy = first_player1Name
print(f"\nTotal Net Points won by {first_player1Name}: {at_net_player1_and_won_by_player1}")
print(f"Percentage of Net Points won by {first_player1Name}: {percentage_at_net_player1_and_won_by_player1:.2f}%")








Serve Results for Kimmi Hance:

Total Serves: 71
First Serve In (Count): 55
First Serve Won (Count): 28
First Serve In (%): 77.46%
First Serve Won (%): 50.91%
Second Serve In (Count): 13
Second Serve Total (Count): 16
Second Serve Won (Count): 5
Second Serve In (%): 81.25%
Second Serve Won (%): 38.46%
Ace (Count): 1
Double Fault (Count): 3
Points Won on Serve (Count) 34
Points Won on Serve (%): 47.89%

Return Results for Kimmi Hance:

Total Return (Count): 68
Return Won (Count): 29
Return Won (%): 42.64705882352941

Return Made (Count): 58
Return Made (%): 0.8529411764705882
Return Error (Count): 10
Return Winner (Count): 0

Deuce Return (Count): 31
Deuce Return Made (Count): 26
Deuce Return Made (%): 0.8387096774193549
Deuce Return Won by Player1 (%): 41.935483870967744
Deuce Return Won by Player1 (Count): 13

Ad Return (Count): 37
Ad Return Made (Count): 32
Ad Return Made (%): 0.8648648648648649
Ad Return Won by Player1 (Count): 16
Ad Return Won by Player1 (%): 43.24324324324324
Deu

In [1168]:
# Filter points where serverName is equal to first_player1Name
filtered_points = point_df[point_df['serverName'] == first_player1Name]

# Group the filtered points by player1ServePlacement and count the occurrences
serve_placement_counts = filtered_points.groupby('player1ServePlacement').size()


# Iterate over filtered_points
for index, point in filtered_points.iterrows():
    serve_placement = point['player1ServePlacement']
    
    # Check if serve placement is not in serve_placement_counts
    if serve_placement not in serve_placement_counts:
        print(point)


# Initialize dictionaries to store counts and percentages
point_won_counts = {}
point_won_percentages = {}
print(f"Total {len(filtered_points)}")

# Iterate over serve placements
for serve_placement, count in serve_placement_counts.items():
    # Filter points with the specific serve placement
    serve_placement_points = filtered_points[filtered_points['player1ServePlacement'] == serve_placement]
    
    # Count points won by first_player1Name
    point_won_count = serve_placement_points[serve_placement_points['pointWonBy'] == first_player1Name].shape[0]
    
    # Calculate percentage
    point_won_percentage = (point_won_count / count) * 100 if count > 0 else 0
    
    # Store counts and percentages
    point_won_counts[serve_placement] = point_won_count
    point_won_percentages[serve_placement] = point_won_percentage

# Print counts and percentages
for serve_placement, count in serve_placement_counts.items():
    print(f"Serve Placement: {serve_placement}")
    print(f"Total Serves: {count}")
    print(f"Serves Won by {first_player1Name}: {point_won_counts.get(serve_placement, 0)}")
    print(f"Percentage: {point_won_percentages.get(serve_placement, 0):.2f}%\n")

pointNumber                                                  31
player1Name                                         Kimmi Hance
player2Name                                      Malaika Rapolu
pointScore                                                  0-0
gameScore                                                   2-3
tiebreakScore                                              None
side                                                      Deuce
serverName                                          Kimmi Hance
returnerName                                     Malaika Rapolu
Position                                              1690313.0
pointEndPosition                                      1705212.0
Duration                                                14899.0
rallyCount                                                    1
rallyCountFreq                                            1 - 4
firstServeIn                                                  0
secondServeIn                           

In [1164]:
print(f"\nError Data for {first_player1Name}:\n")
# Filter the DataFrame based on specified conditions
total_errors = point_df[(point_df['lastShotHitBy'] == first_player1Name) & 
                           (point_df['lastShotResult'] == 'Error')]

import numpy as np

# Filter rows without NaN values in relevant columns
forehand_errors = point_df[(point_df['lastShotHitBy'] == first_player1Name) & 
                           (point_df['lastShotResult'] == 'Error') &
                           (point_df['lastShotFhBh'] == 'Forehand') &
                           (~point_df['errorType'].isnull())]  # Ensure 'errorType' column doesn't have NaN

backhand_errors = point_df[(point_df['lastShotHitBy'] == first_player1Name) & 
                           (point_df['lastShotResult'] == 'Error') &
                           (point_df['lastShotFhBh'] == 'Backhand') &
                           (~point_df['errorType'].isnull())]  # Ensure 'errorType' column doesn't have NaN

# Count the occurrences of 'Forehand' and 'Backhand' separately
forehand_counts = forehand_errors.shape[0]  # Count rows
backhand_counts = backhand_errors.shape[0]  # Count rows

# Print the total error counts for verification
total_error_counts = forehand_counts + backhand_counts

# Get value counts of 'errorType' for Forehand errors
forehand_error_types = forehand_errors['errorType'].value_counts(dropna=False)  # Include NaN values in count

# Get value counts of 'errorType' for Backhand errors
backhand_error_types = backhand_errors['errorType'].value_counts(dropna=False)  # Include NaN values in count

# Print the counts and error types
print("Count of Total errors:", total_error_counts)
print("Count of Forehand errors:", forehand_counts)
print("Count of Backhand errors:", backhand_counts)
print("\nForehand errors:\n", forehand_error_types)
print("\nBackhand errors:\n", backhand_error_types)






# # Group by both 'lastShotDirection' and 'errorType', and then count occurrences
forehand_error_counts = forehand_errors.groupby(['player1LastShotPlacement', 'errorType']).size().unstack(fill_value=0)  # Fill NaN with 0

print("\nValue counts of 'errorType' for Forehand errors with different directions:\n", forehand_error_counts)


# Group by both 'lastShotDirection' and 'errorType', and then count occurrences
backhand_error_counts = backhand_errors.groupby(['player1LastShotPlacement', 'errorType']).size().unstack(fill_value=0)  # Fill NaN with 0

print("\nValue counts of 'errorType' for Backhand errors with different directions:\n", backhand_error_counts)


# forehand_errors = point_df[(point_df['lastShotHitBy'] == first_player1Name) & 
#                            (point_df['lastShotResult'] == 'Error') &
#                            (point_df['lastShotFhBh'] == 'Forehand')]

# backhand_errors = point_df[(point_df['lastShotHitBy'] == first_player1Name) & 
#                            (point_df['lastShotResult'] == 'Error') &
#                            (point_df['lastShotFhBh'] == 'Backhand')]

# # Count the occurrences of 'Forehand' and 'Backhand' separately
# forehand_counts = len(forehand_errors)
# backhand_counts = len(backhand_errors)

# # Print the total error counts for verification
# total_error_counts = len(forehand_errors) + len(backhand_errors)

# # Get value counts of 'errorType' for Forehand errors
# forehand_error_types = forehand_errors['errorType'].value_counts()

# # Get value counts of 'errorType' for Backhand errors
# backhand_error_types = backhand_errors['errorType'].value_counts()

# # Print the counts and error types
# print("Count of Total errors:", total_error_counts)
# print("Count of Forehand errors:", forehand_counts)
# print("Count of Backhand errors:", backhand_counts)
# print("\nForehand errors:\n", forehand_error_types)
# print("\nBackhand errors:\n", backhand_error_types)


# # Group by both 'lastShotDirection' and 'errorType', and then count occurrences
# forehand_error_counts = forehand_errors.groupby(['lastShotDirection', 'errorType']).size()

# print("Value counts of 'errorType' for Forehand errors with different directions:\n", forehand_error_counts)


# # Group by both 'lastShotDirection' and 'errorType', and then count occurrences
# backhand_error_counts = backhand_errors.groupby(['lastShotDirection', 'errorType']).size()


# print("Value counts of 'errorType' for Backhand errors with different directions:\n", backhand_error_counts)


# forehand_errors = point_df[(point_df['lastShotHitBy'] == first_player1Name) & 
#                            (point_df['lastShotResult'] == 'Error') &
#                            (point_df['lastShotFhBh'] == 'Forehand')]

# backhand_errors = point_df[(point_df['lastShotHitBy'] == first_player1Name) & 
#                            (point_df['lastShotResult'] == 'Error') &
#                            (point_df['lastShotFhBh'] == 'Backhand')]

# # Count the occurrences of 'Forehand' and 'Backhand' separately
# forehand_counts = len(forehand_errors)
# backhand_counts = len(backhand_errors)
# total_error_counts = len(total_errors)

# # Get value counts of 'errorType' for Forehand errors
# forehand_error_types = forehand_errors['errorType'].value_counts()

# # Get value counts of 'errorType' for Backhand errors
# backhand_error_types = backhand_errors['errorType'].value_counts()
# print("Count of Total errors:", total_error_counts)
# print("Count of Forehand errors:", forehand_counts)
# print("Count of Backhand errors:", backhand_counts)
# print("\nForehand errors:\n", forehand_error_types)
# print("\nBackhand errors:\n", backhand_error_types)



#


Error Data for Kimmi Hance:

Count of Total errors: 55
Count of Forehand errors: 30
Count of Backhand errors: 25

Forehand errors:
 errorType
Net           12
Long          10
Wide Right     4
Wide Left      4
Name: count, dtype: int64

Backhand errors:
 errorType
Net           15
Long           5
Wide Left      4
Wide Right     1
Name: count, dtype: int64

Value counts of 'errorType' for Forehand errors with different directions:
 errorType                 Long  Net  Wide Left  Wide Right
player1LastShotPlacement                                  
Crosscourt                   1    8          4           1
Down the Line                9    4          0           3

Value counts of 'errorType' for Backhand errors with different directions:
 errorType                 Long  Net  Wide Left  Wide Right
player1LastShotPlacement                                  
Crosscourt                   3    7          1           0
Down the Line                2    8          3           1


In [1143]:
point_df.columns

Index(['pointNumber', 'player1Name', 'player2Name', 'pointScore', 'gameScore',
       'tiebreakScore', 'side', 'serverName', 'returnerName', 'Position',
       'pointEndPosition', 'Duration', 'rallyCount', 'rallyCountFreq',
       'firstServeIn', 'secondServeIn', 'serveResult', 'serveInPlacement',
       'firstServeZone', 'secondServeZone', 'isAce', 'returnDirection',
       'returnFhBh', 'errorType', 'returnError', 'lastShotDirection',
       'lastShotFhBh', 'lastShotHitBy', 'lastShotResult', 'pointWonBy',
       'isBreakPoint', 'atNetPlayer1', 'atNetPlayer2', 'setScore', 'setNum',
       'player1SetScore', 'player2SetScore', 'player1GameScore',
       'player2GameScore', 'player1PointScore', 'player2PointScore',
       'player1TiebreakScore', 'player2TiebreakScore', 'gameNumber',
       'player1ServeResult', 'player1ServePlacement', 'player1ReturnPlacement',
       'player1ReturnFhBh', 'player1LastShotPlacement', 'player1LastShotFhBh',
       'player1LastShotResult', 'Name'],
      d

# Continue down for print to Visuals and print to Json

In [1144]:
first_player1Name = point_df['player1Name'].iloc[0]
print(first_player1Name)

Kimmi Hance


In [1145]:
# Assuming point_df is your DataFrame
total_count = len(point_df[point_df['serverName'] == first_player1Name])
first_serve_in_count = len(point_df[(point_df['serverName'] == first_player1Name) & (point_df['firstServeIn'] == 1)])
percentage_first_serve_in = (first_serve_in_count / total_count) * 100 if total_count > 0 else 0

# Display the results
print("Total Count:", total_count)
print("Count of First Serve In:", first_serve_in_count)
print(f"First Serve In Percentage: {percentage_first_serve_in:.2f}%")


Total Count: 71
Count of First Serve In: 55
First Serve In Percentage: 77.46%


In [1146]:
# Assuming point_df is your DataFrame
total_serves = len(point_df[point_df['serverName'] == first_player1Name])
first_serve_in_count = len(point_df[(point_df['serverName'] == first_player1Name) & (point_df['firstServeIn'] == 1)])
first_serve_won_count = len(point_df[(point_df['serverName'] == first_player1Name) & (point_df['firstServeIn'] == 1) & (point_df['pointWonBy'] == first_player1Name)])
percentage_first_serve_in = (first_serve_in_count / total_serves) * 100 if total_count > 0 else 0
percentage_first_serve_won = (first_serve_won_count / first_serve_in_count) * 100 if first_serve_in_count > 0 else 0

second_serve_total_count = len(point_df[(point_df['serverName'] == first_player1Name) & (point_df['firstServeIn'] == 0)])
second_serve_in_count = len(point_df[(point_df['serverName'] == first_player1Name) & (point_df['firstServeIn'] == 0)& (point_df['secondServeIn'] == 1)])
second_serve_won_count = len(point_df[(point_df['serverName'] == first_player1Name) & (point_df['firstServeIn'] == 0)& (point_df['secondServeIn'] == 1) & (point_df['pointWonBy'] == first_player1Name)])
percentage_second_serve_in = (second_serve_in_count / second_serve_total_count) * 100 if second_serve_total_count > 0 else 0
percentage_second_serve_won = (second_serve_won_count / second_serve_in_count) * 100 if second_serve_in_count > 0 else 0



# Display the results
print("Total Serves:", total_serves)
print("First Serve In (Count):", first_serve_in_count)
print("First Serve Won (Count):", first_serve_won_count)
print(f"First Serve In (%): {percentage_first_serve_in:.2f}%")
print(f"First Serve Won (%): {percentage_first_serve_won:.2f}%")

print("Second Serve In (Count):", second_serve_in_count)
print("Second Serve Total (Count):", second_serve_total_count)
print("Second Serve Won (Count):", second_serve_won_count)
print(f"Second Serve In (%): {percentage_second_serve_in:.2f}%")
print(f"Second Serve Won (%): {percentage_second_serve_won:.2f}%")

# Assuming point_df is your DataFrame
count_is_ace = (point_df[point_df['serverName'] == first_player1Name]['isAce']).sum()
count_is_double_fault = ((point_df['serverName'] == first_player1Name) & (point_df['serveResult'] == "Double Fault")).sum()

# Display the results
print("Ace (Count):", count_is_ace)
print("Double Fault (Count):", count_is_double_fault)

# Count of rows where serverName is equal to the first row of player1Name and pointWonBy is equal to the first row of player1Name
total_service_points_won = len(point_df[(point_df['serverName'] == first_player1Name) & (point_df['pointWonBy'] == first_player1Name)])
total_service_points_won_percentage = total_service_points_won / total_serves *100

# Display the results
print(f"Points Won on Serve (Count) {total_service_points_won}")

print(f"Points Won on Serve (%): {total_service_points_won_percentage:.2f}%")


Total Serves: 71
First Serve In (Count): 55
First Serve Won (Count): 28
First Serve In (%): 77.46%
First Serve Won (%): 50.91%
Second Serve In (Count): 13
Second Serve Total (Count): 16
Second Serve Won (Count): 5
Second Serve In (%): 81.25%
Second Serve Won (%): 38.46%
Ace (Count): 1
Double Fault (Count): 3
Points Won on Serve (Count) 34
Points Won on Serve (%): 47.89%


In [1147]:
point_df.columns

Index(['pointNumber', 'player1Name', 'player2Name', 'pointScore', 'gameScore',
       'tiebreakScore', 'side', 'serverName', 'returnerName', 'Position',
       'pointEndPosition', 'Duration', 'rallyCount', 'rallyCountFreq',
       'firstServeIn', 'secondServeIn', 'serveResult', 'serveInPlacement',
       'firstServeZone', 'secondServeZone', 'isAce', 'returnDirection',
       'returnFhBh', 'errorType', 'returnError', 'lastShotDirection',
       'lastShotFhBh', 'lastShotHitBy', 'lastShotResult', 'pointWonBy',
       'isBreakPoint', 'atNetPlayer1', 'atNetPlayer2', 'setScore', 'setNum',
       'player1SetScore', 'player2SetScore', 'player1GameScore',
       'player2GameScore', 'player1PointScore', 'player2PointScore',
       'player1TiebreakScore', 'player2TiebreakScore', 'gameNumber',
       'player1ServeResult', 'player1ServePlacement', 'player1ReturnPlacement',
       'player1ReturnFhBh', 'player1LastShotPlacement', 'player1LastShotFhBh',
       'player1LastShotResult', 'Name'],
      d

In [1148]:
point_df['lastShotResult'].unique()

array(['Winner', 'Error', None], dtype=object)

In [1149]:
# Assuming point_df is your DataFrame
return_points = point_df[(point_df['returnerName'] == first_player1Name) & (point_df['rallyCount'] >= 2)]

total_return = len(return_points)
returnMade = len(return_points[(return_points['rallyCount'] > 2) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error'))])
returnError = len(return_points[(return_points['lastShotResult'] == 'Error') & (return_points['rallyCount'] == 2)])
returnWinner = len(return_points[(return_points['lastShotResult'] == 'Winner') & (return_points['rallyCount'] == 2)])
returnMadePercentage = returnMade/total_return

returnWonByPlayer1 = len(return_points[return_points['pointWonBy'] == first_player1Name])
returnWonByPlayer1Percentage = returnWonByPlayer1 / total_return * 100 if total_return > 0 else 0

deuceReturnCount = len(return_points[return_points['side'] == 'Deuce'])
adReturnCount = len(return_points[return_points['side'] == 'Ad'])


deuceReturnMade = len(return_points[(return_points['side'] == 'Deuce') & ((return_points['rallyCount'] > 2) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error')))])
adReturnMade = len(return_points[(return_points['side'] == 'Ad') & ((return_points['rallyCount'] > 2) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error')))])

deuceReturnMadePercentage = deuceReturnMade/deuceReturnCount
adReturnMadePercentage = adReturnMade/adReturnCount

deuceReturnWonByPlayer1 = len(return_points[(return_points['side'] == 'Deuce') & (return_points['pointWonBy'] == first_player1Name) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error'))])
adReturnWonByPlayer1 = len(return_points[(return_points['side'] == 'Ad') & (return_points['pointWonBy'] == first_player1Name) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error'))])

deuceReturnWonByPlayer1Percentage = deuceReturnWonByPlayer1 / deuceReturnCount * 100 if deuceReturnCount > 0 else 0
adReturnWonByPlayer1Percentage = adReturnWonByPlayer1 / adReturnCount * 100 if adReturnCount > 0 else 0




print(f"Return Results for {first_player1Name}:\n")

print("Total Return (Count):", total_return)
print("Return Won (Count):", returnWonByPlayer1)
print("Return Won (%):", returnWonByPlayer1Percentage)

print("\nReturn Made (Count):", returnMade)
print("Return Made (%):", returnMadePercentage)
print("Return Error (Count):", returnError)
print("Return Winner (Count):", returnWinner)

print("\nDeuce Return (Count):", deuceReturnCount)
print("Deuce Return Made (Count):", deuceReturnMade)
print("Deuce Return Made (%):", deuceReturnMadePercentage)
print("Deuce Return Won by Player1 (%):", deuceReturnWonByPlayer1Percentage)
print("Deuce Return Won by Player1 (Count):", deuceReturnWonByPlayer1)


print("\nAd Return (Count):", adReturnCount)
print("Ad Return Made (Count):", adReturnMade)
print("Ad Return Made (%):", adReturnMadePercentage)
print("Ad Return Won by Player1 (Count):", adReturnWonByPlayer1)
print("Ad Return Won by Player1 (%):", adReturnWonByPlayer1Percentage)


Return Results for Kimmi Hance:

Total Return (Count): 68
Return Won (Count): 29
Return Won (%): 42.64705882352941

Return Made (Count): 58
Return Made (%): 0.8529411764705882
Return Error (Count): 10
Return Winner (Count): 0

Deuce Return (Count): 31
Deuce Return Made (Count): 26
Deuce Return Made (%): 0.8387096774193549
Deuce Return Won by Player1 (%): 41.935483870967744
Deuce Return Won by Player1 (Count): 13

Ad Return (Count): 37
Ad Return Made (Count): 32
Ad Return Made (%): 0.8648648648648649
Ad Return Won by Player1 (Count): 16
Ad Return Won by Player1 (%): 43.24324324324324


In [1150]:
# Assuming return_points is your DataFrame
deuce_return_points = return_points[(return_points['side'] == 'Deuce') & (return_points['returnerName'] == first_player1Name) & (return_points['rallyCount'] >= 2)]

# Deuce Return Points Separated by returnFhBh
deuce_forehand_return_points = deuce_return_points[deuce_return_points['returnFhBh'] == 'Forehand']
deuce_backhand_return_points = deuce_return_points[deuce_return_points['returnFhBh'] == 'Backhand']


# Count for Deuce Return Points - Made
count_deuce_forehand_made = len(deuce_forehand_return_points[(deuce_forehand_return_points['rallyCount'] > 2) | ((deuce_forehand_return_points['rallyCount'] == 2) & (deuce_forehand_return_points['lastShotResult'] != 'Error'))])
count_deuce_backhand_made = len(deuce_backhand_return_points[(deuce_backhand_return_points['rallyCount'] > 2) | ((deuce_backhand_return_points['rallyCount'] == 2) & (deuce_backhand_return_points['lastShotResult'] != 'Error'))])

# Count for Deuce Return Points - Error
count_deuce_forehand_error = len(deuce_forehand_return_points[(deuce_forehand_return_points['lastShotResult'] == 'Error') & (deuce_forehand_return_points['rallyCount'] == 2)])
count_deuce_backhand_error = len(deuce_backhand_return_points[(deuce_backhand_return_points['lastShotResult'] == 'Error') & (deuce_backhand_return_points['rallyCount'] == 2)])

# Display the counts
print("Deuce Forehand Return Points - Made:", count_deuce_forehand_made)
print("Deuce Forehand Return Points - Error:", count_deuce_forehand_error)

print("\nDeuce Backhand Return Points - Made:", count_deuce_backhand_made)
print("Deuce Backhand Return Points - Error:", count_deuce_backhand_error)


Deuce Forehand Return Points - Made: 14
Deuce Forehand Return Points - Error: 3

Deuce Backhand Return Points - Made: 12
Deuce Backhand Return Points - Error: 2


In [1151]:
# Assuming return_points is your DataFrame
ad_return_points = return_points[(return_points['side'] == 'Ad') & (return_points['returnerName'] == first_player1Name) & (return_points['rallyCount'] >= 2)]

# Ad Return Points Separated by returnFhBh
ad_forehand_return_points = ad_return_points[ad_return_points['returnFhBh'] == 'Forehand']
ad_backhand_return_points = ad_return_points[ad_return_points['returnFhBh'] == 'Backhand']

# Count for Ad Return Points - Made
count_ad_forehand_made = len(ad_forehand_return_points[(ad_forehand_return_points['rallyCount'] > 2) | ((ad_forehand_return_points['rallyCount'] == 2) & (ad_forehand_return_points['lastShotResult'] != 'Error'))])
count_ad_backhand_made = len(ad_backhand_return_points[(ad_backhand_return_points['rallyCount'] > 2) | ((ad_backhand_return_points['rallyCount'] == 2) & (ad_backhand_return_points['lastShotResult'] != 'Error'))])

# Count for Ad Return Points - Error
count_ad_forehand_error = len(ad_forehand_return_points[(ad_forehand_return_points['lastShotResult'] == 'Error') & (ad_forehand_return_points['rallyCount'] == 2)])
count_ad_backhand_error = len(ad_backhand_return_points[(ad_backhand_return_points['lastShotResult'] == 'Error') & (ad_backhand_return_points['rallyCount'] == 2)])

# Display the counts
print("Ad Forehand Return Points - Made:", count_ad_forehand_made)
print("Ad Forehand Return Points - Error:", count_ad_forehand_error)

print("\nAd Backhand Return Points - Made:", count_ad_backhand_made)
print("Ad Backhand Return Points - Error:", count_ad_backhand_error)


Ad Forehand Return Points - Made: 20
Ad Forehand Return Points - Error: 3

Ad Backhand Return Points - Made: 12
Ad Backhand Return Points - Error: 2


In [1152]:
# Total points where atNetPlayer1 = 1
total_at_net_player1 = len(point_df[point_df['atNetPlayer1'] == 1])

# Percentage of points where atNetPlayer1 = 1 out of total points
percentage_at_net_player1 = (total_at_net_player1 / len(point_df)) * 100 if len(point_df) > 0 else 0

# Display the total count and percentage of points where atNetPlayer1 = 1
print(f"Total Net Points for {first_player1Name}: {total_at_net_player1}")
print(f"Percentage of Net Points for {first_player1Name}: {percentage_at_net_player1:.2f}%")

# Points where atNetPlayer1 = 1 and pointWonBy = first_player1Name
at_net_player1_and_won_by_player1 = len(point_df[(point_df['atNetPlayer1'] == 1) & (point_df['pointWonBy'] == first_player1Name)])

# Percentage of points where atNetPlayer1 = 1 and pointWonBy = first_player1Name out of total points where atNetPlayer1 = 1
percentage_at_net_player1_and_won_by_player1 = (at_net_player1_and_won_by_player1 / total_at_net_player1) * 100 if total_at_net_player1 > 0 else 0

# Display the count and percentage of points where atNetPlayer1 = 1 and pointWonBy = first_player1Name
print(f"\nTotal Net Points won by {first_player1Name}: {at_net_player1_and_won_by_player1}")
print(f"Percentage of Net Points won by {first_player1Name}: {percentage_at_net_player1_and_won_by_player1:.2f}%")


Total Net Points for Kimmi Hance: 4
Percentage of Net Points for Kimmi Hance: 2.80%

Total Net Points won by Kimmi Hance: 1
Percentage of Net Points won by Kimmi Hance: 25.00%


In [1153]:
# Filter the DataFrame based on specified conditions
total_errors = point_df[(point_df['lastShotHitBy'] == first_player1Name) & 
                           (point_df['lastShotResult'] == 'Error')]

forehand_errors = point_df[(point_df['lastShotHitBy'] == first_player1Name) & 
                           (point_df['lastShotResult'] == 'Error') &
                           (point_df['lastShotFhBh'] == 'Forehand')]

backhand_errors = point_df[(point_df['lastShotHitBy'] == first_player1Name) & 
                           (point_df['lastShotResult'] == 'Error') &
                           (point_df['lastShotFhBh'] == 'Backhand')]

# Count the occurrences of 'Forehand' and 'Backhand' separately
forehand_counts = len(forehand_errors)
backhand_counts = len(backhand_errors)

# Print the total error counts for verification
total_error_counts = len(forehand_errors) + len(backhand_errors)

# Get value counts of 'errorType' for Forehand errors
forehand_error_types = forehand_errors['errorType'].value_counts()

# Get value counts of 'errorType' for Backhand errors
backhand_error_types = backhand_errors['errorType'].value_counts()

# Print the counts and error types
print("Count of Total errors:", total_error_counts)
print("Count of Forehand errors:", forehand_counts)
print("Count of Backhand errors:", backhand_counts)
print("\nForehand errors:\n", forehand_error_types)
print("\nBackhand errors:\n", backhand_error_types)

# forehand_errors = point_df[(point_df['lastShotHitBy'] == first_player1Name) & 
#                            (point_df['lastShotResult'] == 'Error') &
#                            (point_df['lastShotFhBh'] == 'Forehand')]

# backhand_errors = point_df[(point_df['lastShotHitBy'] == first_player1Name) & 
#                            (point_df['lastShotResult'] == 'Error') &
#                            (point_df['lastShotFhBh'] == 'Backhand')]

# # Count the occurrences of 'Forehand' and 'Backhand' separately
# forehand_counts = len(forehand_errors)
# backhand_counts = len(backhand_errors)
# total_error_counts = len(total_errors)

# # Get value counts of 'errorType' for Forehand errors
# forehand_error_types = forehand_errors['errorType'].value_counts()

# # Get value counts of 'errorType' for Backhand errors
# backhand_error_types = backhand_errors['errorType'].value_counts()
# print("Count of Total errors:", total_error_counts)
# print("Count of Forehand errors:", forehand_counts)
# print("Count of Backhand errors:", backhand_counts)
# print("\nForehand errors:\n", forehand_error_types)
# print("\nBackhand errors:\n", backhand_error_types)


Count of Total errors: 55
Count of Forehand errors: 30
Count of Backhand errors: 25

Forehand errors:
 errorType
Net           12
Long          10
Wide Right     4
Wide Left      4
Name: count, dtype: int64

Backhand errors:
 errorType
Net           15
Long           5
Wide Left      4
Wide Right     1
Name: count, dtype: int64


In [1154]:
# Group by both 'lastShotDirection' and 'errorType', and then count occurrences
forehand_error_counts = forehand_errors.groupby(['lastShotDirection', 'errorType']).size()

print("Value counts of 'errorType' for Forehand errors with different directions:\n", forehand_error_counts)

Value counts of 'errorType' for Forehand errors with different directions:
 lastShotDirection  errorType 
Crosscourt         Long          1
                   Net           8
                   Wide Left     4
                   Wide Right    1
Down the Line      Long          9
                   Net           4
                   Wide Right    3
dtype: int64


In [1155]:

# Group by both 'lastShotDirection' and 'errorType', and then count occurrences
backhand_error_counts = backhand_errors.groupby(['lastShotDirection', 'errorType']).size()

print("Value counts of 'errorType' for Backhand errors with different directions:\n", backhand_error_counts)

Value counts of 'errorType' for Backhand errors with different directions:
 lastShotDirection  errorType 
Crosscourt         Long          3
                   Net           7
                   Wide Left     1
Down the Line      Long          2
                   Net           8
                   Wide Left     3
                   Wide Right    1
dtype: int64


In [1156]:
print(point_df['errorType'].unique())

[None 'Net' 'Long' 'Wide Right' 'Wide Left']


# PRINT POINTCSV VISUALS

In [1157]:
# Save point_df to CSV file

# Assuming point_df is your DataFrame and player1Name and player2Name are the names from the first row
player1NameNoSpace = point_df.iloc[0]['player1Name'].replace(" ", "")
player2NameNoSpace = point_df.iloc[0]['player2Name'].replace(" ", "")

# Save DataFrame to CSV file with modified player names
point_df.to_csv(f'{player1NameNoSpace}_{player2NameNoSpace}_Point_ForVisuals.csv', index=False)


# Remove points

In [1158]:
# Assuming pointdf is a DataFrame

# Display rows where 'isPointStart' is equal to 1 and 'pointStartTime' is NaN in pointdf
missing_start_time = point_df[point_df['Position'].isna() | (point_df['Position'] == "")]
print("Rows with missing start time:")
print(missing_start_time)



Rows with missing start time:
     pointNumber  player1Name     player2Name pointScore gameScore  \
249           34  Kimmi Hance  Malaika Rapolu       0-40       2-3   
339           50  Kimmi Hance  Malaika Rapolu      30-30       3-5   
645           99  Kimmi Hance  Malaika Rapolu       0-15       4-3   

    tiebreakScore   side      serverName    returnerName Position  ...  \
249          None     Ad     Kimmi Hance  Malaika Rapolu     None  ...   
339          None  Deuce  Malaika Rapolu     Kimmi Hance     None  ...   
645          None     Ad     Kimmi Hance  Malaika Rapolu     None  ...   

    player2TiebreakScore gameNumber  player1ServeResult player1ServePlacement  \
249                 None          6        1st Serve In              Ad: Wide   
339                 None          9                None                  None   
645                 None          8        1st Serve In              Ad: Wide   

     player1ReturnPlacement  player1ReturnFhBh player1LastShotPlace

In [1159]:

# Remove those rows from pointdf
point_df_for_viewer = point_df.drop(missing_start_time.index)

In [1160]:
# Change Value Names for Match Viewer Output

# Assuming point_df is your DataFrame and player1Name is the name from the first row
player1Name = point_df_for_viewer.iloc[0]['player1Name']

# Replace values in the 'atNetPlayer1' column
point_df_for_viewer['atNetPlayer1'] = point_df_for_viewer['atNetPlayer1'].replace({0: "", 1: player1Name})

In [1161]:
# Change Na to "" Empty String

# Assuming point_df is your DataFrame
# Replace NaN, None, and NA values with empty strings in point_df
point_df_for_viewer.replace([pd.NA, None, pd.NaT, float('nan')], "", inplace=True)

# Print POINTCSV UPLOAD

In [1162]:
player1Name = point_df_for_viewer.iloc[0]['player1Name'].replace(" ", "")
player2Name = point_df_for_viewer.iloc[0]['player2Name'].replace(" ", "")

# Save DataFrame to CSV file with modified player names
point_df_for_viewer.to_csv(f'{player1Name}_{player2Name}_Point_ForUpload.csv', index=False)
point_df_for_viewer.to_json(f'{player1Name}_{player2Name}_Point_ForUpload.json', orient='records')

In [1163]:
# git fetch origin
# git pull
# git checkoutbranchName
# git add .
# git commit -m "message"
# git push